In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

os.cpu_count()

8

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

full_train = pd.read_csv("datasets/train_clean.csv")

to_drop = ["Name", "Ticket", "Embarked", "PassengerId", *[c for c in full_train if "cabin" in c.lower()]]

full_train.drop(columns=to_drop, inplace=True)

target_col = "Survived"

X = full_train.drop(target_col, axis=1)
y = full_train[target_col]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Standardize the features
scaler = StandardScaler()

# cols = [c for c in X_train if c not in {"Age", "Family_Size", "Fare"}]
# X_train = scaler.fit_transform(X_train[cols])
# X_test = scaler.transform(X_test[cols])

# Define the SVM model
svm_model = SVC()

# Define the parameter grid for GridSearchCV
# param_grid = {
#     'C': [1, 10, 100],
#     'kernel': ['linear'],
#     'gamma': ['scale']
# }

param_grid = {
    'C': [0.1, 1],
    'kernel': ['linear', 'poly', 'sigmoid', 'rbf'],
    'degree': [2, 3, 4],
    'coef0': [1, 10],
    'gamma': ['scale']
}

# param_grid = {
#     'C': [0.001, 0.01, 0.1, 10, 100],
#     'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
#     'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10, 100],
#     'degree': [2, 3, 4, 5],
#     'coef0': [0.0, 0.1, 1, 10]
# }


# Create the GridSearchCV object
grid_search = GridSearchCV(svm_model, param_grid, n_jobs=1, cv=5, scoring='accuracy', verbose=4)

# Fit the model
grid_search.fit(X_train, y_train)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy on the test set
accuracy = accuracy_score(y_test, y_pred)

print(f"Best Parameters: {best_params}")
print(f"Accuracy on Test Set: {accuracy:.4f}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5] END C=0.1, coef0=1, degree=2, gamma=scale, kernel=linear;, score=0.790 total time=   0.0s
[CV 2/5] END C=0.1, coef0=1, degree=2, gamma=scale, kernel=linear;, score=0.810 total time=   0.0s
[CV 3/5] END C=0.1, coef0=1, degree=2, gamma=scale, kernel=linear;, score=0.796 total time=   0.0s
[CV 4/5] END C=0.1, coef0=1, degree=2, gamma=scale, kernel=linear;, score=0.761 total time=   0.0s
[CV 5/5] END C=0.1, coef0=1, degree=2, gamma=scale, kernel=linear;, score=0.768 total time=   0.0s
[CV 1/5] END C=0.1, coef0=1, degree=2, gamma=scale, kernel=poly;, score=0.818 total time=   0.0s
[CV 2/5] END C=0.1, coef0=1, degree=2, gamma=scale, kernel=poly;, score=0.817 total time=   0.0s
[CV 3/5] END C=0.1, coef0=1, degree=2, gamma=scale, kernel=poly;, score=0.810 total time=   0.0s
[CV 4/5] END C=0.1, coef0=1, degree=2, gamma=scale, kernel=poly;, score=0.789 total time=   0.0s
[CV 5/5] END C=0.1, coef0=1, degree=2, gamma=scale, ker

In [3]:
import joblib

joblib.dump(grid_search, 'grid_search_model.joblib')

['grid_search_model.joblib']

In [4]:
loaded_grid_search = joblib.load('grid_search_model.joblib')

In [5]:
for k, v in loaded_grid_search.cv_results_.items():
    print(k, v, sep="\t")

mean_fit_time	[0.00556178 0.00555544 0.01008668 0.00756631 0.0054244  0.00568361
 0.01006122 0.0075182  0.00526428 0.00660644 0.01006641 0.0075397
 0.00557489 0.00674553 0.01001196 0.00868182 0.00589418 0.01174841
 0.00994639 0.008494   0.0058712  0.05720186 0.01092072 0.00998092
 0.00959907 0.00820274 0.01017561 0.00772719 0.00821676 0.01670771
 0.01398673 0.01065879 0.01079345 0.01406999 0.00907078 0.00761518
 0.00805883 0.00860181 0.0088325  0.00666819 0.00712008 0.04877825
 0.00981855 0.00682302 0.00767236 0.29218364 0.00921259 0.00683937]
std_fit_time	[6.64057804e-04 6.61875741e-05 1.25659810e-04 1.32133857e-04
 3.29383101e-04 1.88770454e-04 1.15173637e-04 1.30579712e-04
 2.24567775e-04 2.15966577e-04 1.72398402e-04 1.46062421e-04
 3.06372991e-04 2.23346850e-04 5.75687616e-05 3.37530098e-04
 2.78521927e-04 1.35086016e-03 3.84125998e-05 1.77356028e-04
 2.17824600e-04 1.28432684e-02 8.58990998e-04 2.07796694e-04
 5.23285312e-04 3.95644707e-04 7.25136406e-04 4.06539611e-04
 4.6335909

In [6]:
real_test = pd.read_csv("datasets/test_clean.csv")

In [7]:
real_test.drop(columns=list(set(to_drop)-{"PassengerId"}), inplace=True)

In [8]:
real_test = real_test.interpolate(method='from_derivatives')

In [9]:
pred = pd.DataFrame(best_model.predict(real_test.drop(columns=["PassengerId"]))).rename(columns={0: "Survived"})

In [10]:
pd.concat([real_test[["PassengerId"]], pred], axis=1).to_csv("out_v1.csv", index=None)